Импорт библиотек, загрузка данных

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('events2.csv',  index_col=None)

In [3]:
# Показываю всю свою пробную таблицу для вохзможности проверки результата
df.head(30)

,user_id,event_timestamp,event_name
0,1,1571443200,registration
1,2,1571443200,registration
2,3,1571443200,registration
3,1,1571443200,n
4,4,1571443200,registration
5,5,1571443200,n
6,4,1571443200,n
7,1,1574121600,n
8,10,1574121600,n
9,3,1574121600,n


Перевод даты в формат "год-месяц" и создание соответсвующей колонки

In [4]:
df['date'] = pd.to_datetime(df['event_timestamp'], unit ='s')
df['date'] = df.date.map(lambda x: x.strftime('%Y-%m'))

Создание колонки с датой регистрации для каждого пользователя 

In [5]:
# Функция для ввода даты регистрации в значения колонки
def reg(df):
    if df['event_name'] == 'registration':
        return df['event_timestamp']
    else:
        return None

In [6]:
df['registration'] = df.apply(reg, axis=1)

In [7]:
# Функция ввела данные только для строк, где произошла регитрация, заполняем пробелы, сгруппировав таблицу по юзерам
grp = df.groupby(['user_id'])  
df.registration = grp.registration.apply(lambda x: x.fillna(x.mean()))
df.registration.fillna(df.registration.mean, inplace = True)

Расчет разницы действия с датой регистрации

In [8]:
df['lag'] = df['event_timestamp'] - df['registration']

Сравнение даты возврата с датой регистрации и определение считать ли это действие "возвратом"

In [9]:
def dif(df):
    if (86400 * min_lag) <= df['lag'] <= (86400 * max_lag) and df['user_id'] not in n:
        n.append(df['user_id'])
        return 1
    else:
        return 0

In [10]:
# Кажется логичным взять для расчета возврата не действия в следующем месяце (потому что пользователь
            # мог зарегистрироваться 30 числа, а затем зайти 1 следующего месяца), а какой-то промежуток в днях
# Взял промежуток 20 и 40 дней, то есть месячный промежуток с отклонением 10 в обе стороны - можно как-то изменить
            # в зависимости от конкретной задачи и данных
n = []
min_lag = 20
max_lag = 40
df['return'] = df.apply(dif, axis=1)

Таблица для преобразований для расчета retention

In [11]:
df.head(15)

,user_id,event_timestamp,event_name,date,registration,lag,return
0,1,1571443200,registration,2019-10,1.571443e+09,0.0,0
1,2,1571443200,registration,2019-10,1.571443e+09,0.0,0
2,3,1571443200,registration,2019-10,1.571443e+09,0.0,0
3,1,1571443200,n,2019-10,1.571443e+09,0.0,0
4,4,1571443200,registration,2019-10,1.571443e+09,0.0,0
5,5,1571443200,n,2019-10,1.574122e+09,-2678400.0,0
6,4,1571443200,n,2019-10,1.571443e+09,0.0,0
7,1,1574121600,n,2019-11,1.571443e+09,2678400.0,1
8,10,1574121600,n,2019-11,1.576714e+09,-2592000.0,0
9,3,1574121600,n,2019-11,1.571443e+09,2678400.0,1


Создание таблицы с количеством зарегистрировавшихся пользователей в каждый месяц

In [12]:
reg = df.pivot_table(index='date', columns='event_name', values='user_id', aggfunc=lambda x: len(x.unique())).reset_index()
names = ['date', 'registration']
reg = reg[names]
reg.head(5)

event_name,date,registration
0,2019-10,4.0
1,2019-11,3.0
2,2019-12,3.0
3,2020-01,NaN


Создание таблицы с количеством вернувшихся на следующий месяц пользователей

In [13]:
retrn = df.pivot_table(index='date', columns='return', values='user_id', aggfunc=lambda x: len(x.unique())).reset_index()
retrn.rename(columns={1: 'return'}, inplace=True)
names = ['date', 'return']
retrn = retrn[names]
retrn['return'] = retrn['return'].shift(-1) # Важный момент, что нужно поднять данные на одну строку, так как возврат происходит
                                            # в следующий месяц
retrn.head()

return,date,return
0,2019-10,2.0
1,2019-11,1.0
2,2019-12,2.0
3,2020-01,NaN


Итоговая таблица, как требовалось по заданию

In [14]:
table = reg.join(retrn.set_index('date'), on='date')
table['retention'] = (table['return'] / table['registration'])*100
table.head()

,date,registration,return,retention
0,2019-10,4.0,2.0,50.000000
1,2019-11,3.0,1.0,33.333333
2,2019-12,3.0,2.0,66.666667
3,2020-01,NaN,NaN,NaN
